Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/automated-machine-learning/manymodels/02_Training/02_Training_Pipeline.png)

# トレーニング パイプライン - 自動 ML
_**自動機械学習を使用した Many models のトレーニング**_

---

このノートブックでは、自動機械学習を使用して 11,973 のモデルをトレーニングおよび登録する方法を示します。11,973 モデルのトレーニングプロセスを並列化するために AutoMLPipelineBuilder を利用します。このノートブックでは、オレンジジュースの販売データセットを使用して、各ブランドと各店舗のオレンジジュースの販売量を予測しています。データの詳細については、データ準備ノートブックを参照してください。

<span style="color:red"><b>注: ワークスペースごとに並列実行できる実行の数には制限があり、現時点ではワークスペースごとに並列処理を 1 つの実験ごとに最大 20 回に設定することをお勧めします。ユーザーがより多くの並列処理を必要とし、この制限を増やした場合、要求数過多数エラー (HTTP 429) が発生する可能性があります。 </b></span>

<span style="color:red"><b> AutoML の依存関係が一貫していることを確認するために、最新バージョンの SDK を使用してください。</b></span>

In [ ]:
#!pip install --upgrade azureml-sdk[automl]

Many models に必要な azureml-contrib-automl-automl-pipeline-steps パッケージをインストールします。

In [ ]:
# !pip install azureml-contrib-automl-pipeline-steps

### 前提条件

この時点で、次の内容が完了している必要があります：

1. [00_Setup_AML_Workspace notebook](../../00_Setup_AML_Workspace.ipynb) を使用して、AML ワークスペースが作成されていること
2. [01_Data_Preparation.ipynb](../../01_Data_Preparation.ipynb) を実行してデータセットを作成していること

## 1.0 ワークスペース、データストア、実験の設定

In [ ]:
import azureml.core
from azureml.core import Workspace, Datastore
import pandas as pd

# ワークスペースのセットアップ
ws= Workspace.from_config()

# ワークスペースの確認
ws.get_details()


# データストアのセットアップ
dstore = ws.get_default_datastore()

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Default datastore name'] = dstore.name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

### 実験の選択

In [ ]:
from azureml.core import Experiment

experiment = Experiment(ws, 'manymodels-training-pipeline')

print('Experiment name: ' + experiment.name)

## 2.0 登録済みの FileDatasets を呼び出す

ここでは 11,973 のデータセットと AutoMLPipelineBuilder を使用して、11,973 の時系列を構築し、各店舗ブランドの数量を予測します。

各データセットは、7 つの列と 122 行を含むブランドの 2 年間のオレンジジュース売上データを表します。

まず、ワークスペースにデータセットを登録する必要があります。データの準備ノートブックでは、2 つのデータセットをワークスペースに登録する方法を示します。

登録済みの 'oj_data_small' FileDatasets には、最初の 10 個の csv ファイルが含まれ、'oj_data' には 11,973 個の csv ファイルがすべて含まれます。以下のセルでは filedatasets_10_models_input または filedatasets_all_models_inputs のどちらを AutoMLPipelineBuilder に渡すかを選択できます。

**最初に filedatasets_10_models** で試して、すべてが正常に実行されていることを確認してから、filedatasets_all_modelsにスケールアップすることをお勧めします。

In [ ]:
from azureml.core.dataset import Dataset

filedst_10_models = Dataset.get_by_name(ws, name='oj_data_small_train')
filedst_10_models_input = filedst_10_models.as_named_input('train_10_models')

#filedst_all_models = Dataset.get_by_name(ws, name='oj_data_train')
#filedst_all_models_inputs = filedst_all_models.as_named_input('train_all_models')

## 3.0 トレーニング パイプラインの構築
データセット、ワークスペースおよびデータストアが設定できたので、トレーニング用のパイプラインにまとめることができます。

### コンピュート ターゲットの選択

現在 AutoMLPipelineBuilder は AMLCompute のみをサポートしています。失敗した場合は、別のコンピュート クラスターに変更できます。

以下は AutoMLPipelineBuilder に渡すコンピュート ターゲットです。

In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

# クラスタの名前を選択
amlcompute_cluster_name = "cpucluster"

found = False
# ワークスペースにこのコンピュート ターゲットが存在しているか確認
cts = ws.compute_targets
if amlcompute_cluster_name in cts and cts[amlcompute_cluster_name].type == 'AmlCompute':
    found = True
    print('既存のコンピュート ターゲットがあります。')
    compute = cts[amlcompute_cluster_name]
    
if not found:
    print('新しいコンピュート ターゲットを作成しています...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D16S_V3',
                                                           min_nodes=2,
                                                           max_nodes=20)
    # クラスタの作成
    compute = ComputeTarget.create(ws, amlcompute_cluster_name, provisioning_config)
    
print('クラスタの状態を確認中...')
# 最小ノード数と特定のタイムアウトをポーリングできます。
# min_node_count が提供されていない場合は、クラスターのスケール設定が使用されます。
compute.wait_for_completion(show_output = True, min_node_count = None, timeout_in_minutes = 20)
    
# 現在の AmlCompute ステータスの詳細なビューは、get_status() を使用して確認できます。

## トレーニング

このディクショナリは [AutoML 設定](https://docs.microsoft.com/ja-jp/python/api/azureml-train-automl-client/azureml.train.automl.automlconfig.automlconfig?view=azure-ml-py#parameters)を定義し、この予測タスクの時間列の名前と最大予測期間を追加します。

|プロパティ|説明|
|-|-|
|**task**|タスクは予測（forecasting）を指定します|
|**primary_metric**|これは最適化するメトリックです。<br> 予測では、次のプライマリ指標がサポートされます。 <br><i>spearman_correlation</i><br><i>normalized_root_mean_squared_error</i><br><i>r2_score</i><br><i>normalized_mean_absolute_error</i>|
|**blocked_models**|blocked_model で指定されたモデルは AutoML で使用されません。サポートされているすべてのモデルは[こちら](https://docs.microsoft.com/ja-jp/python/api/azureml-train-automl-client/azureml.train.automl.constants.supportedmodels.forecasting?view=azure-ml-py)からご覧いただけます。|
|**iterations**|トレーニングするモデルの数。これはオプションですが、シナリオによっては重要な制御内容となります。|
|**iteration_timeout_minutes**|モデルをトレーニングできる最大時間 (分単位) です。これはオプションであり、データセットによって異なります。データセットのトレーニングに必要な時間を帖佐の上設定してください。OJ データセットの場合は、20 分に設定します。|
|**experiment_timeout_hours**|実験が終了するまでにかかる最大時間を時間単位で指定できます。|
|**label_column_name**|ラベル列の名前。|
|**n_cross_validations**|交差検証の分割数。ローリングオリジン検証は、時系列を一定の方法で分割するために使用されます。|
|**enable_early_stopping**|スコアが短期的に改善していない場合に早期終了を有効にするためのフラグです。|
|**time_column_name**|時刻列データの列名。|
|**max_horizon**|過去のトレーニング データから予測したい期間数。期間はデータから推測されます。|
|**grain_column_names**|同じタイムスタンプを持つ複数の行を持つデータ内の時系列を一意に識別するために使用される列名。|
|**partition_column_names**|モデルをグループ化するために使用する列の名前です。時系列の場合、グループは個々の時系列を分割してはなりません。つまり、各グループには、1 つ以上の時系列が含まれている必要があります。|
|**track_child_runs**|子の実行の追跡を無効にするフラグです。フラグが False に設定されている場合は、Best run (メトリックとモデル) のみが追跡されます。|
|**pipeline_fetch_max_batch_size**|トレーニングのために一度に取得するパイプライン (トレーニング アルゴリズム) の数を指定し、大規模なトレーニング時のスロットリングを減らすことができます。|

In [ ]:
import logging

partition_column_names = ['Store', 'Brand']

automl_settings = {
    "task" : 'forecasting',
    "primary_metric" : 'normalized_root_mean_squared_error',
    "iteration_timeout_minutes" : 10, # これはデータセットに基づいて変更する必要があります。この値を設定するまでにトレーニングにかかる時間を確認するようお願いします
    "iterations" : 15,
    "experiment_timeout_hours" : 1,
    "label_column_name" : 'Quantity',
    "n_cross_validations" : 3,
    "verbosity" : logging.INFO, 
    "debug_log": 'automl_oj_sales_debug.txt',
    "time_column_name": 'WeekStarting',
    "max_horizon" : 20,
    "track_child_runs": False,
    "partition_column_names": partition_column_names,
    "grain_column_names": ['Store', 'Brand'],
    "pipeline_fetch_max_batch_size": 15
}

### Many modelsトレーニングステップを構築する

AutoMLPipelineBuilder は Many models のトレーニング ステップを構築するために使用されます。ユース ケースに適したワーカーとノードの数を決定する必要があります。process_count_per_node は、コンピュート VM のコア数に基づいています。node_count は、使用するマスターノードの数を決定し、ノード数を増やすとトレーニングプロセスがスピードアップします。

* <b>experiment</b>: 現在の実験。

* <b>automl_settings</b>: 自動 ML 設定ディクショナリ。

* <b>train_data</b>: トレーニングするデータセット。

* <b>compute_target</b>: トレーニング用のコンピュート ターゲット。

* <b>partition_column_names</b>: パーティション列名。

* <b>node_count</b>: ユーザー スクリプトの実行に使用するコンピューティング ノードの数。3から始めることをお勧めしまうが、トレーニング時間が長すぎる場合には node_countを増やします。

* <b>process_count_per_node</b>: ノードあたりのプロセス数。

* <b>run_invocation_timeout</b>: run() メソッドの呼び出しタイムアウト (秒単位)。タイムアウトは、AutoML の実行の最大トレーニング時間 (及び多少のバッファ) 内に設定する必要があります。デフォルトは 60 秒です。

* <b>output_datastore</b>: トレーニング結果を出力する出力用データストアを指定します。

* <b>train_env(オプション)</b>: トレーニングに使用するトレーニング環境定義をオプションで提供できます。

<span style="color:red"><b>注: ワークスペースごとに並列実行できる実行の数には制限があり、現在は、ワークスペースごとに並列処理を 1 つの実験あたり最大 320 回に設定することをお勧めします。ユーザーがより多くの並列処理を必要とし、この制限を増やす場合、要求数過多エラー (HTTP 429) が発生する可能性があります。 </b></span>

In [ ]:
#!pip install azureml.contrib.automl.pipeline.steps

In [ ]:
from azureml.contrib.automl.pipeline.steps import AutoMLPipelineBuilder

train_steps = AutoMLPipelineBuilder.get_many_models_train_steps(experiment=experiment,
                                                                automl_settings=automl_settings,
                                                                train_data=filedst_10_models_input,
                                                                compute_target=compute,
                                                                partition_column_names=partition_column_names,
                                                                node_count=2,
                                                                process_count_per_node=8,
                                                                run_invocation_timeout=3700,
                                                                output_datastore=dstore)

## 4.0 トレーニング パイプラインの実行

### 実行するパイプラインを送信する

次に、実行するパイプラインを送信します。トレーニング パイプライン全体は、現在の AutoMLPipelineBuilder 設定で 1台の STANDARD_D16S_V3 VM を使用して約1時間11分かかります。

In [ ]:
from azureml.pipeline.core import Pipeline
#from azureml.widgets import RunDetails

pipeline = Pipeline(workspace=ws, steps=train_steps)
run = experiment.submit(pipeline)
#RunDetails(run).show()

jupyter ノートブックでトレーニング プロセスを監視する場合は、以下のコマンドを実行します。トレーニング中にログをライブストリーミングします。

**注**: このコマンドはコンピューティング VM（旧Notebook VM）では動作しない場合もありますが、ローカル ラップトップでは動作します。

In [ ]:
run.wait_for_completion(show_output=True)

正常にトレーニングされ、自動MLモデルが登録されました。

## 5.0 トレーニング パイプラインの出力をレビューする

トレーニング パイプラインは、モデルをトレーニングし、ワークスペースに登録します。トレーニング済みモデルは、Azure Machine Learning Studio の 'Models' の中で確認できます。トレーニングに問題がある場合は、パイプライン実行の中にある実行された 'many-models-training' の中にある 'Logs' 内のログで実行状態を確認することができます。
詳細は logs/user/worker/<ip> の中にある stdout と stderr をご覧ください


## 6.0 登録されたモデル名およびタグと共に AutoML 実行の一覧を取得する

次のコード スニペットは、実験のすべての automl 実行を反復処理し、詳細を一覧表示します。

**Framework** - AutoML, **Dataset** - 入力データセット, **Run** - AutoML 実行ID, **Status** - AutoML 実行ステータス,  **Model** - 登録モデル名, **Tags** - モデルのタグ, **StartTime** - 開始時間, **EndTime** - 終了時刻, **ErrorType** - ErrorType, **ErrorCode** - ErrorCode, **ErrorMessage** - エラーメッセージ

In [ ]:
from scripts.helper import get_training_output
import os

training_results_name = "training_results"
training_output_name = "many_models_training_output"

training_file = get_training_output(run, training_results_name, training_output_name)
all_columns = ["Framework", "Dataset", "Run", "Status", "Model", "Tags", "StartTime", "EndTime" , "ErrorType", "ErrorCode", "ErrorMessage" ]
df = pd.read_csv(training_file, delimiter=" ", header=None, names=all_columns)
training_csv_file = "training.csv"
df.to_csv(training_csv_file)
print("Training output has", df.shape[0], "rows. Please open", os.path.abspath(training_csv_file), "to browse through all the output.")

## 7.0 パイプラインの発行とスケジュール (オプション)

### 7.1 パイプラインを発行する

満足できるパイプラインを作成したら、パイプラインを発行して、後からプログラムで呼び出すことができます。パイプラインの発行と呼び出しの詳細については、この[チュートリアル](https://docs.microsoft.com/ja-jp/azure/machine-learning/how-to-create-machine-learning-pipelines#publish-a-pipeline)を参照してください。

In [ ]:
# published_pipeline = pipeline.publish(name = 'automl_train_many_models',
#                                      description = 'train many models',
#                                      version = '1',
#                                      continue_on_step_failure = False)

### 7.2 パイプラインのスケジュール実行
また、時間ベースまたは変更ベースのスケジュールで実行するように[パイプラインをスケジュール](https://docs.microsoft.com/ja-jp/azure/machine-learning/how-to-trigger-published-pipeline)することもできます。これは、毎月、またはデータドリフトなどの別のトリガーに基づいて、モデルを自動的に再トレーニングするために使用できます。

In [ ]:
# from azureml.pipeline.core import Schedule, ScheduleRecurrence
    
# training_pipeline_id = published_pipeline.id

# recurrence = ScheduleRecurrence(frequency="Month", interval=1, start_time="2020-01-01T09:00:00")
# recurring_schedule = Schedule.create(ws, name="automl_training_recurring_schedule", 
#                             description="Schedule Training Pipeline to run on the first day of every month",
#                             pipeline_id=training_pipeline_id, 
#                             experiment_name=experiment.name, 
#                             recurrence=recurrence)

## 8.0 ワークスペースの保守(オプション)

### 8.1 実行中の全ての実行をキャンセルする

特定の実験で実行され続けている実行をキャンセルします。

In [ ]:
# from scripts.helper import cancel_runs_in_experiment
# failed_experiment =  'ここを変更して、実験名を入力してください'
# # 次のスクリプトは、この実験内で現在実行されているすべての実行をキャンセルします
# cancel_runs_in_experiment(ws, failed_experiment)